LangChainMemory
```
LangChainMemory : 이전 대화 내용을 기억해서 문맥을 유지하는 역할 LangChain 0.3X부터는 LCEL 기반으로 체인을 구성,
RunnableWithMessageHistoru, ChatMessageHistory 등의 컴퍼넌트를 활용해서 세션별 대화 기록을 관리, 대화가 장기화될 경우 요약 메모리를 도입해서 과거 대화를 LLM으로 요약하고 축약된 형태로 저장해서 프롬프트의 길이 문제를 해결결
```

In [1]:
%pip install langchain langchain-openai python-dotenv --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [3]:
from langchain_core.chat_history import InMemoryChatMessageHistory
# 메모리 객체 생성
history = InMemoryChatMessageHistory()
history.add_user_message('언녕하세요 제 이름은 이주영영입니다.')
history.add_ai_message('안녕하세요 이주영님, 무엇을 도와드릴까요?')
# 현재까지의 대화 내용 확인
for msg in history.messages:
    print(f'{msg.type} : {msg.content}')

human : 언녕하세요 제 이름은 이주영영입니다.
ai : 안녕하세요 이주영님, 무엇을 도와드릴까요?


In [4]:
# Radis 기반 채팅 기록 저장소

from langchain_redis import RedisChatMessageHistory
import os
REDIS_URL = os.getenv('REDIS_URL', 'redis://localhost:6379')
session_id = 'user_123'

history = RedisChatMessageHistory(session_id = session_id, redis_url = REDIS_URL)
history.add_user_message('안녕하세요 제 이름은 홍길동 입니다.')
history.add_ai_message('안녕하세요 홍길동님, 무엇을 도와드릴까요?')

# 현재까지의 대화 내용 확인
for msg in history.messages:
    print(f'{msg.type} : {msg.content}')


ModuleNotFoundError: No module named 'langchain_redis'

In [7]:
# 세션기반 다중사용자 메모리 구조 구현 - 다중사용자 챗복
# 핵심: session_id를 키로 하는 메모리 저장소 만들고 사용자의 대화는 키별로 저장한다
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
# 프롬프트
prompt = ChatPromptTemplate.from_messages([
    ("system",'당신은 뛰어난 한국어 상담 챗봇입니다. 질문에 친절하고 자세히 답변해주세요'),
    #history 키로 전달된 메세지 목록은 chain 실행시 해당 위치에 넣겠다는 의미
    MessagesPlaceholder(variable_name='history'), 
    ('human','{input}')
])
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0) 

In [8]:
# LCEL
from langchain_core.output_parsers import StrOutputParser
chain = prompt | llm | StrOutputParser()

In [5]:
# 세션별 메모리 저장소를 딕셔너리로 만들고, 존재하지 않는 새로운 세션 id가 들어오면 InMemoryChatMessageHistory를 생성
# get_sessino_history 를 구현

In [14]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
#세션 id -> 대화 기록 객체 매핑
store = {}
def get_session_history(session_id : str) -> InMemoryChatMessageHistory:
    '''"세션 ID"에 해당하는 대화 기록 객체를 반환합니다. (없으면 새로 생성)'''
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 메모리를 통합한 체인 래퍼생성
chatbot = RunnableWithMessageHistory(
    runnable = chain,
    get_session_history = get_session_history,
    input_messages_key= 'input',
    history_messages_key= 'history'
)

In [15]:
# 두개의 세션을 번갈아가면서 대화 RunnableWithMessageHistory 가 각 세션에 맞는 대화 기록을 관리합니다
sessions = ['user_a','user_b']
questions = [
    '안녕하세요, 저는 홍길동입니다. 당신은 누구신가요?', #user_a 첫번째 질문
    '안녕하세요, 저는 이순신입니다. 당신은 어떤 일을 하시나요?', #user_b 첫 번째 질문
    '저는 프로그래밍을 배우고 있습니다. 당신은 어떤 일을 하시나요?', #user_a 두 번째 질문
    '저는 역사에 관심이 많습니다. 당신은 어떤 분야에 관심이 있나요?' #user_b 두 번째 질문
]
for i,question in enumerate(questions):
    session_id = sessions[i%2] # 세션 ID를 번갈아가며 사용 
    result = chatbot.invoke({'input':question}, config = {'configurable': {'session_id':session_id}})
    print(f'[{session_id}] 질문: {question}')
    print(f'[{session_id}] 챗봇: {result}\n')

[user_a] 질문: 안녕하세요, 저는 홍길동입니다. 당신은 누구신가요?
[user_a] 챗봇: 안녕하세요, 홍길동님! 저는 여러분의 질문에 답변하고 도움을 드리기 위해 만들어진 챗봇입니다. 어떤 궁금한 점이나 도움이 필요하신 부분이 있으신가요?

[user_b] 질문: 안녕하세요, 저는 이순신입니다. 당신은 어떤 일을 하시나요?
[user_b] 챗봇: 안녕하세요, 이순신님! 저는 여러분의 질문에 답변하고, 다양한 정보와 도움을 제공하는 챗봇입니다. 궁금한 점이나 도움이 필요한 부분이 있다면 언제든지 말씀해 주세요!

[user_a] 질문: 저는 프로그래밍을 배우고 있습니다. 당신은 어떤 일을 하시나요?
[user_a] 챗봇: 프로그래밍을 배우고 계시다니 멋지네요! 저는 여러분의 질문에 답변하고, 정보 제공, 문제 해결, 그리고 다양한 주제에 대한 상담을 하는 역할을 하고 있습니다. 프로그래밍에 관련된 질문이나 도움이 필요하시면 언제든지 말씀해 주세요! 어떤 언어를 배우고 계신가요?

[user_b] 질문: 저는 역사에 관심이 많습니다. 당신은 어떤 분야에 관심이 있나요?
[user_b] 챗봇: 역사에 관심이 많으시다니 정말 멋지네요! 저는 특정한 관심 분야가 없지만, 다양한 주제에 대한 정보를 제공할 수 있습니다. 역사, 과학, 기술, 문화, 예술 등 여러 분야에 대해 이야기할 수 있으니, 궁금한 점이나 더 알고 싶은 주제가 있다면 말씀해 주세요!



In [16]:
result = chatbot.invoke({'input':"저는 철수예요, 반갑습니다"}, config={'configurable' : {'session_id' : 'user_c'}})
print(f'[user_c] 질문: 저는 철수예요, 반갑습니다.')
print(f'[user_c] 챗봇: {result}\n')

[user_c] 질문: 저는 철수예요, 반갑습니다.
[user_c] 챗봇: 안녕하세요, 철수님! 반갑습니다. 어떻게 도와드릴까요? 궁금한 점이나 이야기하고 싶은 것이 있다면 말씀해 주세요.



In [17]:
result = chatbot.invoke({'input' : '저는 누구라고요?'}, config = {'configurable': {'session_id' : 'user_c'}})
result
                        

'철수님이라고 하셨습니다! 맞나요? 더 궁금한 점이나 이야기하고 싶은 것이 있다면 언제든지 말씀해 주세요.'

요약 메모리 구현(대화내용 자동 요약)
```
긴 대화내용을 모두 프롬프트에 기록하는 것은 비 효율적 -> 프롬프트에 길이 제한에 걸릴 가능성이 있음
Conversation Summary Memory
0.3x 버전에서는 직접 요약용 체인을 만들어서 ChatMessageHistory에 적용
```
어떻게 요약?
```
- 일정길이 이상으로 대화가 누적되면, 과거 대화를 요약해서 핵심내용만 남김
- 요약결과를 메모리에 시스템 메시지 등으로 저장 -> 메모리 절약
- 새로운 사용자 입력 시 요약된 맥락 + 최근 몇 메시지만 참고해서 llm 전달
```

In [19]:
# 요약용 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_messages([
    ('system','당신은 대화 요약 전문가입니다. 대화의 주요 내용을 간결하게 요약해 주세요'),
    ('human','{conversation}')   # 잔체 대화 내용을 하나의 문자열로 전달
])
# LCEL  
summary_chain = summary_prompt | llm | StrOutputParser()

In [20]:
# user_d 세션에 대화내용을 기록 간 대화 생성
long_queries = [
    '안녕, 오늘 우리 뭐하려고 했지?',
    '아 맞다 내일 회의자료 준비해야지, 회의는 몇시지?',
    '그 회의에 누가 참석하는지 기억나니?',
    '단위 프로젝트 진행 상황도 공유해야 할까?',
    '최근에 이야기 했던 새로운 기능에 대한 업데이트는 있어?'
]
session_id = 'user_d'
for q in long_queries:
    answer = chatbot.invoke({'input':q}, config={'configurable' : {'session_id' : session_id}})

print(f'요약전 user_d의 메모리 메세지 개수: {len(store[session_id].messages)}')
print(store[session_id]) #요약하기전 대화 내용


요약전 user_d의 메모리 메세지 개수: 10
Human: 안녕, 오늘 우리 뭐하려고 했지?
AI: 안녕하세요! 오늘 어떤 이야기를 나누고 싶으신가요? 궁금한 점이나 하고 싶은 주제가 있다면 말씀해 주세요. 함께 이야기해보아요!
Human: 아 맞다 내일 회의자료 준비해야지, 회의는 몇시지?
AI: 회의 시간이 정확히 몇 시인지 기억이 나지 않으신가요? 회의 일정은 보통 이메일이나 캘린더에 기록되어 있을 텐데, 확인해보시면 좋을 것 같아요. 만약 회의 준비에 도움이 필요하시다면 어떤 자료를 준비해야 하는지 말씀해 주시면 도와드릴 수 있습니다!
Human: 그 회의에 누가 참석하는지 기억나니?
AI: 회의에 참석하는 사람들을 기억하는 것은 중요하지만, 제가 그 정보를 알 수는 없어요. 보통 회의 초대장이나 이메일에 참석자 목록이 포함되어 있을 텐데, 그걸 확인해보시면 좋을 것 같아요. 참석자 목록을 알고 계신다면, 그들과의 논의 주제나 역할에 대해 이야기해드릴 수 있습니다! 도움이 필요하시면 말씀해 주세요.
Human: 단위 프로젝트 진행 상황도 공유해야 할까?
AI: 네, 단위 프로젝트 진행 상황을 공유하는 것은 좋은 아이디어입니다. 회의에서 프로젝트의 현재 상태, 진행 중인 작업, 완료된 작업, 그리고 앞으로의 계획 등을 공유하면 참석자들이 프로젝트에 대한 이해를 높이고, 필요한 피드백이나 지원을 받을 수 있습니다. 

또한, 진행 상황을 공유함으로써 팀원 간의 협업을 촉진하고, 문제를 조기에 발견하여 해결할 수 있는 기회를 제공할 수 있습니다. 어떤 내용을 포함할지 고민 중이라면, 구체적인 진행 상황이나 이슈, 다음 단계 등을 정리해보세요. 도움이 필요하시면 언제든지 말씀해 주세요!
Human: 최근에 이야기 했던 새로운 기능에 대한 업데이트는 있어?
AI: 새로운 기능에 대한 업데이트는 프로젝트의 진행 상황에 따라 다를 수 있습니다. 만약 그 기능에 대한 구체적인 내용이나 개발 상황을 알고 계신다면, 그 정보를 바탕으로 업데이트를 준비하시면 좋을 것 같

In [26]:
# 전체 대화 내용을 요약하고 마지막 사용자 질문-답변 쌍만 원본 유지
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
# 요약 대상 대화 내용 추출(마지막 QA 쌍 제외한 이전 내용)
message = store[session_id].messages

# 1번 주고받은 대화 이후에 또 했다는 것 -> 마지막을 빼야함
if len(message) > 2 :
    original_dialog = '\n'.join([f'{msg.type.upper()} : {msg.content}' for msg in message[ : -2]])
else : 
    original_dialog = '\n'.join([f'{msg.type.upper()} : {msg.content}' for msg in message])

# llm으로 요약 생성
summary_text = summary_chain.invoke({'conversation' : original_dialog})
print("== 요약 내용 ==")
print(summary_text)
# 기존 메모리를 요약으로 교체 : 이전 내용 요약본 + 최근 QA 유지
new_history = InMemoryChatMessageHistory()
new_history.messages.append(SystemMessage(content = f'요약: {summary_text}'))

# 최근 대화의 마지막 QA쌍 복원
if len(message) >= 2 :
    last_user_msg = message[-2]
    last_ai_msg = message[-1]
    # 휴먼 메시지인지 검사
    if isinstance(last_user_msg, HumanMessage) :
        new_history.add_user_message(last_user_msg.content)
    else :
        new_history.messages.append(last_user_msg)
    if isinstance(last_ai_msg, AIMessage) :
        new_history.add_ai_message(last_ai_msg.content)
    else :
        new_history.messages.append(last_ai_msg)
# 메모리 교체
store[session_id] = new_history
    


== 요약 내용 ==
대화 요약: 
사용자는 내일 회의 자료를 준비해야 한다고 언급하며 회의 시간과 참석자를 확인하려고 한다. AI는 회의 일정과 참석자 목록을 이메일이나 캘린더에서 확인할 것을 제안하고, 단위 프로젝트 진행 상황을 회의에서 공유하는 것이 좋다고 조언한다. AI는 진행 상황 공유가 팀 협업과 문제 해결에 도움이 될 것이라고 강조하며, 필요한 경우 추가 도움을 제공할 준비가 되어 있다고 말한다.
